# Confidence dynamics

In [1]:
import glob
import pandas as pd
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import column
from bokeh.models import Band, ColumnDataSource, NumeralTickFormatter, HoverTool
from bokeh.models.widgets import RadioButtonGroup, RadioGroup
from bokeh.palettes import Spectral11, Category20b_12
output_notebook()

Loading BokehJS ...

In [2]:
def fix_uncertainty(results):
    results = [res.rename(index=str, columns={"uncertainty":"std"}) for res in results]
    for res in results:
        res['var'] = res['std'] ** 2
    return results

In [3]:
fns = glob.glob("../results/confidence_dynamics/*.msgpack")
results = [pd.read_msgpack(fn) for fn in fns]
results = fix_uncertainty(results)
results[0].head(5)

,step,std,qsa,var
0,1,0.093037,s0a0,0.008656
1,1,0.098914,s0a1,0.009784
2,1,0.105727,s1a0,0.011178
3,1,0.090196,s1a1,0.008135
4,1,0.113695,s2a0,0.012927


In [4]:
fns

['../results/confidence_dynamics/2018Nov14_183006_mdp6_n10_p05_vote1.msgpack',
 '../results/confidence_dynamics/2018Nov14_182922_mdp6_n20_p09_vote1.msgpack',
 '../results/confidence_dynamics/2018Nov14_183116_mdp6_n20_p05_vote1.msgpack',
 '../results/confidence_dynamics/2018Nov13_163619_mdp6_n10_p09_vote1.msgpack']

In [5]:
tooltips=[("(step,σ2)", "(@step, @var{0.0000}"),
          ("(step,σ)", "(@step, @std{0.0000})"),
          ("qsa", "@qsa"),
]

def get_plot(data):
    data_plt = figure(title="Data",
                      plot_height=400,
                      plot_width=900,
                      y_axis_type="linear",
                      # y_range=(0,1000000),
                      background_fill_color='#ffffff',
                      tooltips=tooltips)

    data_plt.yaxis.formatter=NumeralTickFormatter(format="0.0000")

    for series, color in zip(data['qsa'].unique(), Category20b_12):
        df = data.loc[data['qsa'] == series]
        data_plt.line(x='step', y='var', legend=series, source=df,
                      line_width=3, color=color, alpha=0.8)
    data_plt.legend.location = "top_left"
    data_plt.legend.click_policy="hide"
    return data_plt

In [7]:
plt = get_plot(results[1])
show(plt)

In [8]:
fns = glob.glob("../results/confidence_dynamics/2018Nov15_142122_mdp6_n20_p09_vote1/*.msgpack")
trials = [pd.read_msgpack(fn) for fn in fns]
trials = fix_uncertainty(trials)
data = pd.concat(trials, ignore_index=True)
print(f'{len(trials)} trials.')
data.head(10)

100 trials.


,step,std,qsa,trial,var
0,1,0.156303,s0a0,53,0.024431
1,1,0.152399,s0a1,53,0.023226
2,1,0.164209,s1a0,53,0.026965
3,1,0.118511,s1a1,53,0.014045
4,1,0.110801,s2a0,53,0.012277
5,1,0.126837,s2a1,53,0.016088
6,1,0.194959,s3a0,53,0.038009
7,1,0.143358,s3a1,53,0.020552
8,1,0.153693,s4a0,53,0.023621
9,1,0.140208,s4a1,53,0.019658


In [9]:
data.sample(10)

,step,std,qsa,trial,var
443485,522,0.010140,s0a1,39,1.028274e-04
247275,289,0.033042,s1a1,62,1.091756e-03
28301,380,0.005100,s2a1,32,2.601050e-05
95547,56,0.045328,s1a1,35,2.054655e-03
517578,854,0.000742,s3a0,72,5.505183e-07
442307,423,0.030093,s5a1,39,9.055663e-04
541025,431,0.003950,s2a1,70,1.560092e-05
249325,460,0.008257,s0a1,62,6.817386e-05
662944,322,0.083198,s2a0,99,6.921954e-03
719453,745,0.001968,s2a1,93,3.874964e-06


In [10]:
def transform(data, groupby, transform='mean', cap=False):
    if cap:
        print(cap)
        data = data.loc[data['step'] <= cap]
    if transform == 'mean':
        return data.groupby(groupby).mean().reset_index()
    else:
        raise ValueError

In [15]:
data = transform(data, ['step', 'qsa'], cap=800)
data.sample(5)

800


,step,qsa,std,var
510,43,s3a0,0.080266,0.008511
2096,175,s4a0,0.112924,0.014508
1138,95,s5a0,0.091643,0.009657
3152,263,s4a0,0.110952,0.014374
4714,393,s5a0,0.041599,0.002332


In [16]:
plt = get_plot(data)
show(plt)